In [1]:
import os

In [2]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\Sleep Detection\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\Sleep Detection'

In [5]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/AbhayBisht0801/SleepDetect-MLflow-DVC-End2End.mlflow' 
os.environ['MLFLOW_TRACKING_USERNAME']='AbhayBisht0801'
os.environ['MLFLOW_TRACKING_PASSWORD']='dae5a0dsdsd2c6d887749ffe32bcacasa2454512451646464'


In [6]:
import tensorflow as tf
model=tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluateConfig:
    path_of_model:Path
    testing_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int

In [8]:
from src.Sleep_Detection.constants import *
from src.Sleep_Detection.utils.common import read_yaml,create_directories,save_json


In [9]:
class ConigurationManagaer:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_evaluation_config(self) ->EvaluateConfig:
        eval_config=EvaluateConfig(path_of_model='artifacts/training/model.h5',
                                   testing_data='artifacts\\data_ingestion\\dataset\\test',
                                   mlflow_uri='https://dagshub.com/AbhayBisht0801/SleepDetect-MLflow-DVC-End2End.mlflow',
                                   all_params=self.params,
                                   params_batch_size=self.params.BATCH_SIZE,
                                   params_image_size=self.params.IMAGE_SIZE
                                   )
        return eval_config
        

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self,config=EvaluateConfig):
        self.config=config
    def test_generator(self):
        datagenerator = dict(
            rescale = 1./255
            
        )

        dataflow = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator
        )

        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.testing_data,
            shuffle=False,
            **dataflow
        )
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    def evaluation(self):
        self.model=self.load_model(self.config.path_of_model)
        self.test_generator()
        self.score=self.model.evaluate(self.test_generator)
    def save_score(self):
        scores={"loss":self.score[0],'accuracy':self.score[1]}
        save_json(path=Path('score.json'),data=scores)
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":

                
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")



In [12]:
try:
    config=ConigurationManagaer()
    eval_config=config.get_evaluation_config()
    evaluation=Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-02-12 23:17:45,503: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-02-12 23:17:45,533: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-12 23:17:45,536: INFO: common: created directory at: artifacts]
Found 91 images belonging to 2 classes.
6/6 [==============================] - 11s 2s/step - loss: 0.3137 - accuracy: 0.8901


2024/02/12 23:18:00 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-02-12 23:18:01,775: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\bisht\AppData\Local\Temp\tmp1fxwlj_n\model\data\model\assets
[2024-02-12 23:18:02,416: INFO: builder_impl: Assets written to: C:\Users\bisht\AppData\Local\Temp\tmp1fxwlj_n\model\data\model\assets]


c:\Users\bisht\OneDrive\Desktop\Sleep Detection\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/02/12 23:18:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
